LLaVA-1.6をnotebookで動かす方法は現状、こちらのYoutubeしか見つからない
- https://www.youtube.com/watch?v=SEavari8xaU

LLaVA-1.5であれば、HuggingFaceがサポートしているのですぐ動かせそう
- https://colab.research.google.com/drive/1qsl6cd2c8gGtEW1xV5io7S8NHh-Cp1TV?usp=sharing#scrollTo=DFVZgElEQk3x

### LLaVA-1.5を動かす

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import requests
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
from transformers import BitsAndBytesConfig
from transformers import pipeline
import openai
from openai import OpenAI
from src.utils import download_image_from_s3

openai.api_key = ''

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16
# )

model_id = "/media/sj-archimedes/data/03_pretrained_model/llm/llava-hf/llava-1.5-7b-hf"

pipe = pipeline(
    "image-to-text",
    model=model_id,
    # device='cuda:1',
    device_map='auto',
    # model_kwargs={"quantization_config": quantization_config}
)

In [ ]:
# image_url = "https://llava-vl.github.io/static/images/view.jpg"
# image = Image.open(requests.get(image_url, stream=True).raw)
# image

df = pd.read_pickle('/media/sj-archimedes/data/share/OddAI_Library_practice/data08/lap_trainval_20221001-20231031.pkl')
df = df.query('creative_type == "image"')
df['creative_media_url'] = df['creative_media_url'].map(lambda x: x[0])
df['creative_media_hash'] = df['creative_media_hash'].map(lambda x: x[0])

In [ ]:
# 日本語で生成されるとうまくいかない例。このような例が散見されたため、日本語を直接出力する方法は断念する
# df = df.query('creative_media_hash == "QRBZ4C3P6UTEJR62P2XAGIDZ6Q"')

# s3_url = df['creative_media_url'].iloc[0]

# img = download_image_from_s3(s3_url)
# image_list = [img]
# img
s3_url = df['creative_media_url'].sample(3).tolist()
print(s3_url)
image_list = [download_image_from_s3(path) for path in s3_url]

In [ ]:
plt.subplots(1,3, figsize=(15,5))
for i, img in enumerate(image_list):
    plt.subplot(1,3,i+1)
    plt.imshow(img)
plt.show()

In [ ]:
%%time
max_new_tokens = 200
# prompt = "USER: <image>\nPlease describe this image in Japanese.\nASSISTANT:"
# prompt = "USER: <image>\nこの画像を日本語で説明してください。\nASSISTANT:"
# prompt = "USER: <image>\nPlease describe this image.\nASSISTANT:"
prompt = "USER: <image>\nPlease explain in detail how this advertisement image has been designed to appear attractive to consumers.\nASSISTANT:"

for img in image_list:
    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.show()
    outputs = pipe(
        img,
        prompt=prompt,
        generate_kwargs={"max_new_tokens": 512}
    )
    
    response = outputs[0]['generated_text'].split('\nASSISTANT: ')[1]
    
    print(response)
    
    instruction = f"次の文章を日本語に翻訳してください。\n{response}"
    
    
    client = OpenAI()
    
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": instruction},
      ]
    )
    
    response = response.choices[0].message.content
    
    print(response)
    print('\n==========\n')

In [ ]:
%%time

max_new_tokens = 200
# prompt = "USER: <image>\nPlease describe this image in Japanese.\nASSISTANT:"
prompt = "USER: <image>\nPlease describe this image.\nASSISTANT:"

for img in image_list:
    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.show()
    outputs = pipe(
        img,
        prompt=prompt,
        generate_kwargs={"max_new_tokens": 200}
    )
    
    response = outputs[0]['generated_text'].split('\nASSISTANT: ')[1]
    
    print(response)
    
    instruction = f"次の文章を日本語に翻訳してください。\n{response}"    
    client = OpenAI()
    
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": instruction},
      ]
    )
    
    response = response.choices[0].message.content
    
    print(response)
    print('\n==========\n')

In [ ]:
%%time

max_new_tokens = 200
prompt = "USER: <image>\nPlease describe this advertisement image.\nASSISTANT:"

for img in image_list:
    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.show()
    outputs = pipe(
        img,
        prompt=prompt,
        generate_kwargs={"max_new_tokens": 200}
    )
    
    response = outputs[0]['generated_text'].split('\nASSISTANT: ')[1]
    
    print(response)
    
    instruction = f"次の文章を日本語に翻訳してください。\n{response}"    
    client = OpenAI()
    
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": instruction},
      ]
    )
    
    response = response.choices[0].message.content
    
    print(response)
    print('\n==========\n')

In [ ]:
%%time

max_new_tokens = 200
prompt = "USER: <image>\nPlease explain the layout representation of this advertisement image.\nASSISTANT:"

for img in image_list:
    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.show()
    outputs = pipe(
        img,
        prompt=prompt,
        generate_kwargs={"max_new_tokens": 200}
    )
    
    response = outputs[0]['generated_text'].split('\nASSISTANT: ')[1]
    
    print(response)
    
    instruction = f"次の文章を日本語に翻訳してください。\n{response}"    
    client = OpenAI()
    
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": instruction},
      ]
    )
    
    response = response.choices[0].message.content
    
    print(response)
    print('\n==========\n')

In [ ]:
%%time

max_new_tokens = 200
prompt = "USER: <image>\nPlease explain the layout techniques used, particularly regarding the material images and text layers in the advertisement image.\nASSISTANT:"

for img in image_list:
    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.show()
    outputs = pipe(
        img,
        prompt=prompt,
        generate_kwargs={"max_new_tokens": 200}
    )
    
    response = outputs[0]['generated_text'].split('\nASSISTANT: ')[1]
    
    print(response)
    
    instruction = f"次の文章を日本語に翻訳してください。\n{response}"    
    client = OpenAI()
    
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": instruction},
      ]
    )
    
    response = response.choices[0].message.content
    
    print(response)
    print('\n==========\n')

# バッチ